## Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# ------- commutation matrix -------
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]

# ------- matrix A(Sigma) -------
def create_A_Sigma(cov: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))

# ------- matrix A (intervention) -------
def create_A(A_Sigma: np.ndarray, mean: np.ndarray, p: int) -> np.ndarray:
    identity = mean[0] * np.eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * np.eye(p)))
    return np.vstack((A_Sigma, identity))

num_exp = 10000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix
b = 2
reg_param = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]

## Experiments

### Ex. 0
<img src="Graphics/graph40.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.

### Ex. 1
<img src="Graphics/graph41.png" alt="Drawing" style="width: 150px;"/>

In [2]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 7, 8])
supp_compl = np.array([2, 5, 6])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_1 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_1.to_pickle("Experiments/DLL_nonsimple_1.pkl")

### Ex. 2
<img src="Graphics/graph42.png" alt="Drawing" style="width: 150px;"/>

In [3]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [2, 0], [2, 1]])
support = np.array([0, 1, 3, 4, 5, 8])
supp_compl = np.array([2, 6, 7])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_2 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_2.to_pickle("Experiments/DLL_nonsimple_2.pkl")

### Ex. 3
<img src="Graphics/graph43.png" alt="Drawing" style="width: 150px;"/>

In [4]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 2]])
support = np.array([0, 1, 3, 4, 6, 7, 8])
supp_compl = np.array([2, 5])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_3 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_3.to_pickle("Experiments/DLL_nonsimple_3.pkl")

### Ex. 4
<img src="Graphics/graph44.png" alt="Drawing" style="width: 150px;"/>

In [5]:
p = 3
seed = 0
index = 0

index_zero = np.array([[1, 2], [2, 0]])
support = np.array([0, 1, 2, 3, 4, 7, 8])
supp_compl = np.array([5, 6])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_4 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_4.to_pickle("Experiments/DLL_nonsimple_4.pkl")

### Ex. 5
<img src="Graphics/graph45.png" alt="Drawing" style="width: 150px;"/>

In [6]:
p = 3
seed = 0
index = 0

index_zero = np.array([[2, 0], [2, 1]])
support = np.array([0, 1, 2, 3, 4, 5, 8])
supp_compl = np.array([6, 7])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_5 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_5.to_pickle("Experiments/DLL_nonsimple_5.pkl")

### Ex. 6
<img src="Graphics/graph46.png" alt="Drawing" style="width: 150px;"/>

In [7]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 5, 7, 8])
supp_compl = np.array([2, 6])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_6 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_6.to_pickle("Experiments/DLL_nonsimple_6.pkl")

### Ex. 7
<img src="Graphics/graph47.png" alt="Drawing" style="width: 150px;"/>

In [8]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2]])
support = np.array([0, 1, 3, 4, 5, 6, 7, 8])
supp_compl = np.array([2])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 8), dtype=object), columns=["True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 8]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)
        temp2.iloc[index, i + 1] = frob

experiment_nonsimple_7 = temp2[["Intervention", "True M", "p=0.1", "p=0.05", "p=0.01", "p=0.005", "p=0.001", "p=0.0005", "p=0.0001"]]
experiment_nonsimple_7.to_pickle("Experiments/DLL_nonsimple_7.pkl")

### Ex. 0
<img src="Graphics/graph48.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.